# 12-03: 适配器模式与网关模式适配器模式和网关模式是系统集成中常用的设计模式。

In [ ]:
// ========== 1. 适配器模式 (Adapter Pattern) ==========
// 目标接口
interface MediaPlayer {
  play(audioType: string, fileName: string): void;
}
// 已存在的类 (需要适配)
class AdvancedMediaPlayer {
  playVlc(fileName: string): void {
    console.log(`Playing vlc file: ${fileName}`);
  }

  playMp4(fileName: string): void {
    console.log(`Playing mp4 file: ${fileName}`);
  }
}
// 适配器
class MediaAdapter implements MediaPlayer {
  private advancedPlayer: AdvancedMediaPlayer;

  constructor() {
    this.advancedPlayer = new AdvancedMediaPlayer();
  }

  play(audioType: string, fileName: string): void {
    if (audioType === 'vlc') {
      this.advancedPlayer.playVlc(fileName);
    } else if (audioType === 'mp4') {
      this.advancedPlayer.playMp4(fileName);
    }
  }
}
// 使用适配器的客户端
class AudioPlayer implements MediaPlayer {
  private adapter: MediaAdapter | null = null;

  play(audioType: string, fileName: string): void {
    if (audioType === 'mp3') {
      console.log(`Playing mp3 file: ${fileName}`);
    } else if (audioType === 'vlc' || audioType === 'mp4') {
      this.adapter = new MediaAdapter();
      this.adapter.play(audioType, fileName);
    } else {
      console.log(`Invalid media. ${audioType} format not supported`);
    }
  }
}
// 使用
const player = new AudioPlayer();
player.play('mp3', 'song.mp3');
player.play('mp4', 'movie.mp4');
player.play('vlc', 'video.vlc');
player.play('avi', 'video.avi'); // 不支持

In [ ]:
// ========== 2. 实际应用: API 适配器 ==========
// 第三方 API 接口 (旧版)
interface OldPaymentAPI {
  makePayment(amount: number, currency: string): Promise<string>;
  checkStatus(transactionId: string): Promise<string>;
}
// 新的统一支付接口
interface PaymentProcessor {
  processPayment(payment: PaymentRequest): Promise<PaymentResult>;
  getTransactionStatus(id: string): Promise<TransactionStatus>;
}

interface PaymentRequest {
  amount: number;
  currency: string;
  description?: string;
  metadata?: Record<string, unknown>;
}

interface PaymentResult {
  success: boolean;
  transactionId: string;
  timestamp: Date;
}

interface TransactionStatus {
  id: string;
  status: 'pending' | 'completed' | 'failed';
  amount: number;
  currency: string;
}
// 适配器实现
class OldPaymentAdapter implements PaymentProcessor {
  constructor(private oldApi: OldPaymentAPI) {}

  async processPayment(payment: PaymentRequest): Promise<PaymentResult> {
    const txId = await this.oldApi.makePayment(
      payment.amount,
      payment.currency
    );

    return {
      success: true,
      transactionId: txId,
      timestamp: new Date()
    };
  }

  async getTransactionStatus(id: string): Promise<TransactionStatus> {
    const status = await this.oldApi.checkStatus(id);
    return {
      id,
      status: this.mapStatus(status),
      amount: 0, // 旧 API 不返回金额
      currency: 'USD'
    };
  }

  private mapStatus(status: string): TransactionStatus['status'] {
    switch (status) {
      case 'success': return 'completed';
      case 'fail': return 'failed';
      default: return 'pending';
    }
  }
}

In [ ]:
// ========== 3. 网关模式 (Gateway Pattern) ==========
// 统一的 API 网关接口
interface APIGateway {
  request<T>(service: string, endpoint: string, options?: RequestInit): Promise<T>;
  authenticate(credentials: Credentials): Promise<AuthToken>;
  rateLimitCheck(clientId: string): boolean;
}

interface Credentials {
  apiKey: string;
  secret?: string;
}

interface AuthToken {
  token: string;
  expiresAt: Date;
}
// 网关实现
class ServiceGateway implements APIGateway {
  private services: Map<string, string> = new Map();
  private authTokens: Map<string, AuthToken> = new Map();
  private rateLimiter: RateLimiter;

  constructor() {
    this.rateLimiter = new RateLimiter(100); // 100 requests per minute
  }

  registerService(name: string, baseUrl: string): void {
    this.services.set(name, baseUrl);
  }

  async request<T>(
    service: string,
    endpoint: string,
    options?: RequestInit
  ): Promise<T> {
    const baseUrl = this.services.get(service);
    if (!baseUrl) {
      throw new Error(`Service ${service} not found`);
    }

    const response = await fetch(`${baseUrl}${endpoint}`, {
      ...options,
      headers: {
        'Content-Type': 'application/json',
        ...options?.headers
      }
    });

    if (!response.ok) {
      throw new Error(`Request failed: ${response.statusText}`);
    }

    return response.json();
  }

  async authenticate(credentials: Credentials): Promise<AuthToken> {
    // 验证 API Key
    const token: AuthToken = {
      token: `token_${Date.now()}`,
      expiresAt: new Date(Date.now() + 3600000) // 1 hour
    };

    this.authTokens.set(credentials.apiKey, token);
    return token;
  }

  rateLimitCheck(clientId: string): boolean {
    return this.rateLimiter.checkLimit(clientId);
  }
}

// 简单的速率限制器
class RateLimiter {
  private requests: Map<string, number[]> = new Map();

  constructor(private maxRequests: number) {}

  checkLimit(clientId: string): boolean {
    const now = Date.now();
    const windowStart = now - 60000; // 1 minute window

    const clientRequests = this.requests.get(clientId) || [];
    const recentRequests = clientRequests.filter(t => t > windowStart);

    if (recentRequests.length >= this.maxRequests) {
      return false;
    }

    recentRequests.push(now);
    this.requests.set(clientId, recentRequests);
    return true;
  }
}

In [ ]:
// ========== 4. 微内核架构 ==========
// 插件接口
interface Plugin {
  readonly name: string;
  readonly version: string;
  initialize(core: Core): void;
  destroy(): void;
}

// 核心系统
interface Core {
  registerHook(hookName: string, callback: Function): void;
  unregisterHook(hookName: string, callback: Function): void;
  emitHook(hookName: string, ...args: any[]): Promise<any[]>;
  getService<T>(name: string): T | undefined;
  registerService<T>(name: string, service: T): void;
}
// 核心实现
class MicroKernel implements Core {
  private plugins: Map<string, Plugin> = new Map();
  private hooks: Map<string, Function[]> = new Map();
  private services: Map<string, any> = new Map();

  loadPlugin(plugin: Plugin): void {
    if (this.plugins.has(plugin.name)) {
      throw new Error(`Plugin ${plugin.name} already loaded`);
    }

    plugin.initialize(this);
    this.plugins.set(plugin.name, plugin);
    console.log(`Plugin ${plugin.name} v${plugin.version} loaded`);
  }

  unloadPlugin(name: string): void {
    const plugin = this.plugins.get(name);
    if (plugin) {
      plugin.destroy();
      this.plugins.delete(name);
      console.log(`Plugin ${name} unloaded`);
    }
  }

  registerHook(hookName: string, callback: Function): void {
    const hooks = this.hooks.get(hookName) || [];
    hooks.push(callback);
    this.hooks.set(hookName, hooks);
  }

  unregisterHook(hookName: string, callback: Function): void {
    const hooks = this.hooks.get(hookName) || [];
    const index = hooks.indexOf(callback);
    if (index > -1) {
      hooks.splice(index, 1);
    }
  }

  async emitHook(hookName: string, ...args: any[]): Promise<any[]> {
    const hooks = this.hooks.get(hookName) || [];
    const results = [];

    for (const hook of hooks) {
      try {
        const result = await hook(...args);
        results.push(result);
      } catch (error) {
        console.error(`Hook ${hookName} failed:`, error);
      }
    }

    return results;
  }

  registerService<T>(name: string, service: T): void {
    this.services.set(name, service);
  }

  getService<T>(name: string): T | undefined {
    return this.services.get(name);
  }
}
// 示例插件
class LoggingPlugin implements Plugin {
  readonly name = 'logging';
  readonly version = '1.0.0';

  initialize(core: Core): void {
    core.registerHook('beforeRequest', (req) => {
      console.log(`[Request] ${req.method} ${req.url}`);
    });

    core.registerHook('afterRequest', (req, res) => {
      console.log(`[Response] ${res.statusCode}`);
    });
  }

  destroy(): void {
    console.log('Logging plugin destroyed');
  }
}
// 使用
const kernel = new MicroKernel();
kernel.loadPlugin(new LoggingPlugin());